In [17]:
import importlib
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

 load tokenizer
 class dataset: return input id and output id(input id + 1)
  dataloader

In [48]:
class dataset_V1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input = []
        self.output = []
        
        # txt_ids = tokenizer.encode(txt) 
        txt_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(txt_ids) - max_length, stride):
            input_chunk = txt_ids[i: i+max_length]
            output_chunk = txt_ids[i+1: i+max_length+1] 
            self.input.append(torch.tensor(input_chunk))
            self.output.append(torch.tensor(input_chunk))

    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]
    

In [49]:
def dataloader_v1(txt, batch_size, max_length, stride, 
                  shuffle = True, drop_last = True, num_workers = 0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    dataset = dataset_V1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
    )
    return dataloader
    
    

In [57]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(len(raw_text))
tokenizer = tiktoken.get_encoding("gpt2")
print(len(tokenizer.encode(raw_text)))
print(tokenizer.encode(raw_text)[:20])
print(tokenizer.decode(tokenizer.encode(raw_text)[:20]))
print(raw_text[:20])

20479
5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438]
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--
I HAD always thought


In [74]:
max_length=64
dataloader = dataloader_v1(raw_text, batch_size=8, max_length=64, stride=128)
iter_data = iter(dataloader)
print(next(iter_data))

[tensor([[19672,   484,   550,   262,  1295,   286, 15393,   438, 16706,   262,
          4318,  6103,   287,   257, 14005,  7872,   393,  4808, 13698, 10322,
          6532,    62,  8263,    12,  3823,    11,   393,   257, 36364,  1396,
           417,  4624,   523,   326,   340,  1718,   262,  1657,   832, 41160,
           286,  1468,  9932,   316,   666,   966,    13,   383,   517, 12949,
          1295,  2627,   262,  4286,  1365,    26,  1865,    11,   355,   616,
          2951,  6348, 23840,   284],
        [  423,  4750,   326,  9074,    13,   402,   271, 10899,   373,  5527,
            26,   290,   340,   373,  3393, 34953,   856,   326,   607,  5229,
           373, 37895,   422,   428, 25179,   257, 19217,   475,  8904, 14676,
            13,   632,   318,    11,   355,   257,  3896,    11,   262,   661,
           508, 40987,  1637,   508,   651,   749,   503,   286,   340,    26,
           290,  3619,   338, 19992, 31564,   286,   465,  3656,   338,  1263,
          523

In [76]:
vocab_size = 50257
output_dim = 256
context_length = 1024
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
position_embedding_layer = torch.nn.Embedding(context_length, output_dim)

max_length=64
for batch in dataloader:
    x, y = batch
    
    token_embedding = token_embedding_layer(x) # batch_size, seq_length, embedding_dim

    position_embedding = position_embedding_layer(torch.arange(max_length)) 
    final_embedding = token_embedding + position_embedding

final_embedding.shape


torch.Size([8, 64, 256])